# Load extracted data from npy-files - WIP

This is work in progress.

Since there are overlapping areas and we do not want to have duplicates in our dataset it makes sense to store the data again with the duplicates removed.
To do so we derived the distance to the raster border and the coordinates in the extraction steps.
With this information we need a logic to descide from which tile to select a pixel or polygons given that they exist multiple times in overlapping areas.
It makes sense to take a sample from the tile where it is further away from the border.

In [189]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# - - - - - - - - - - - - - - - - - - - - 
# DEFAULT IMPORTS - IN ALL NOTEBOKS
from src import configs

prjconf = configs.ProjectConfigParser()

# - - - - - - - - - - - - - - - - - - - - 
# NOTEBOOK SPECIFIC IMPORTS
import numpy as np
from pathlib import Path
import pandas as pd
import rasterio
from tqdm import tqdm

from eobox.raster import load_extracted
from eobox.raster.extraction import convert_df_to_geodf


tilenames = prjconf.get("Params", "tiles").split(" ")
tilenames

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


['32UNU', '32UPU', '32UQU', '33UUP']

## Inputs

### Parameters

In [198]:
scoll_id = 1
scoll_name = f"scoll{scoll_id:02d}"

paths_ref_vector = {}
dirs_extracted = {}
for tile in tilenames: 
    print(tile, "")
    paths_footprint = prjconf.get_path("Raw", "tile_footprint", tile=tile)
    paths_ref_vector[tile] = prjconf.get_path("Interim", "clc_subset5", tile=tile)
    dirs_extracted[tile] = prjconf.get_path("Processed", "extracted", tile=tile) / paths_ref_vector[tile].stem
    print(dirs_extracted[tile])

32UNU 
/home/ben/Devel/Projects/classify-hls/data/processed/L3/extracted/32UNU/clc2018_32UNU_subset_500000
32UPU 
/home/ben/Devel/Projects/classify-hls/data/processed/L3/extracted/32UPU/clc2018_32UPU_subset_500000
32UQU 
/home/ben/Devel/Projects/classify-hls/data/processed/L3/extracted/32UQU/clc2018_32UQU_subset_500000
33UUP 
/home/ben/Devel/Projects/classify-hls/data/processed/L3/extracted/33UUP/clc2018_33UUP_subset_500000


In [307]:
target_crs = {'init': 'epsg:4326'}
for i, tile in enumerate(tilenames[2::]):
    print(tile)
    dir_extracted = dirs_extracted[tile]
    print(dir_extracted)
    df = load_extracted(dir_extracted, 
                        ['aux_vector_pid.npy', 
                         'aux_coord_*.npy',
                         'aux_raster_dist2rb']).astype("int")
    df = convert_df_to_geodf(df, crs=dir_extracted).to_crs(target_crs)
    df["tile"] = tile
    df.drop(["aux_coord_x", "aux_coord_x"], axis=1)
    print("shape df", df.shape)
    if i == 0:
        gdf_aux = df
        del df
    else:
        gdf_aux = gdf_aux.append(df)
        del df
    
    print("shape gdf", gdf_aux.shape)
display(gdf_aux.head())
display(gdf_aux.tail())

32UQU
/home/ben/Devel/Projects/classify-hls/data/processed/L3/extracted/32UQU/clc2018_32UQU_subset_500000
shape df (1574401, 5)
shape gdf (1574401, 5)
33UUP
/home/ben/Devel/Projects/classify-hls/data/processed/L3/extracted/33UUP/clc2018_33UUP_subset_500000
shape df (1640453, 5)
shape gdf (3214854, 5)


,aux_vector_pid,aux_coord_x,aux_coord_y,geometry,tile
0,1,700785,5399985,POINT (11.73001205413447 48.72052780083798),32UQU
1,1,700815,5399985,POINT (11.73041947146679 48.72051814003508),32UQU
2,1,700845,5399985,POINT (11.73082688858274 48.72050847779305),32UQU
3,1,700875,5399985,POINT (11.73123430548229 48.72049881411188),32UQU
4,1,700905,5399985,POINT (11.7316417221654 48.72048914899158),32UQU


,aux_vector_pid,aux_coord_x,aux_coord_y,geometry,tile
1640448,17251,409605,5290215,POINT (13.79379716388919 47.7589629921921),33UUP
1640449,17251,409635,5290215,POINT (13.79419738299685 47.75896719811678),33UUP
1640450,17251,409665,5290215,POINT (13.79459760219567 47.75897140264617),33UUP
1640451,17251,409695,5290215,POINT (13.79499782148565 47.75897560578029),33UUP
1640452,17251,409725,5290215,POINT (13.79539804086673 47.75897980751913),33UUP
